In [19]:
#
#  Creating a pyomo parest version of example from James Rawlings Reactor Design Book
#  https://github.com/rawlings-group/paresto/blob/master/examples/green_book/hbv_det.m
#

In [20]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyomo.environ as pyo
import pyomo.dae as dae
import scipy.stats as spstat
from pyomo.contrib.interior_point.inverse_reduced_hessian import inv_reduced_hessian_barrier

In [21]:
data_df = pd.read_csv('hbv_paresto_data.csv', sep = '\t')
data_df.head()

,t,ca_exp,cb_exp,cc_exp
0,0,0.89606,0.0000,0.00
1,2,0.70478,2.9690,336.04
2,4,0.51757,5.5807,264.59
3,6,0.63263,6.9731,330.36
4,8,0.80145,10.3210,343.77


In [22]:
data = [{'ca_exp': {k: v for (k, v) in zip(data_df.t, data_df.ca_exp)},
         'cb_exp': {k: v for (k, v) in zip(data_df.t, data_df.cb_exp)},
         'cc_exp': {k: v for (k, v) in zip(data_df.t, data_df.cc_exp)}
        }]

In [23]:
def hbv_model(data):
    
    ca_exp = data['ca_exp']
    cb_exp = data['cb_exp']
    cc_exp = data['cc_exp']
    texp = list(ca_exp.keys())
    
    ca0 = 1.0
    cb0 = 0.0
    cc0 = 0.0
    
    m = pyo.ConcreteModel()
    
    m.k1 = pyo.Var(initialize = 6.3, bounds = (1e-3, 10))
    m.k2 = pyo.Var(initialize = 0.07, bounds = (1e-3, 10))
    m.k3 = pyo.Var(initialize = 1412, bounds = (100, 5000))
    m.k4 = pyo.Var(initialize = 0.17, bounds = (1e-3, 10))
    m.k5 = pyo.Var(initialize = 0.69, bounds = (1e-3, 10))
    m.k6 = pyo.Var(initialize = 3.5e-6, bounds = (1e-6, 1e-5))
    
    m.time = dae.ContinuousSet(bounds = (0, max(texp)), initialize = texp)
    
    m.ca = pyo.Var(m.time, initialize = ca0)
    m.cb = pyo.Var(m.time, initialize = cb0)
    m.cc = pyo.Var(m.time, initialize = cc0)
    
    m.dca = dae.DerivativeVar(m.ca)
    m.dcb = dae.DerivativeVar(m.cb)
    m.dcc = dae.DerivativeVar(m.cc)
    
    def _dca_eq(m, t):
        if t == 0:
            return pyo.Constraint.Skip
        else:
            return m.dca[t] == m.k2 * m.cb[t] - m.k4 * m.ca[t]
    m.dca_eq = pyo.Constraint(m.time, rule = _dca_eq)

    def _dcb_eq(m, t):
        if t == 0:
            return pyo.Constraint.Skip
        else:
            return m.dcb[t] == m.k1 * m.ca[t] - m.k2 * m.cb[t] - m.k6 * m.cb[t] * m.cc[t]
    m.dcb_eq = pyo.Constraint(m.time, rule = _dcb_eq)
    
    def _dcc_eq(m, t):
        if t == 0:
            return pyo.Constraint.Skip
        else:
            return m.dcc[t] == m.k3 * m.ca[t] - m.k5 * m.cc[t] - m.k6 * m.cb[t] * m.cc[t]
    m.dcc_eq = pyo.Constraint(m.time, rule = _dcc_eq)

    def _initcon(m):
        yield m.ca[m.time.first()] == ca0
        yield m.cb[m.time.first()] == cb0
        yield m.cc[m.time.first()] == cc0
    m.initcon = pyo.ConstraintList(rule = _initcon)
        
    def wssq_rule(m):
        return sum((m.ca[t] - ca_exp[t]) ** 2 + 
                   1e-2 * (m.cb[t] - cb_exp[t]) ** 2 + 
                   1e-4 * (m.cc[t] - cc_exp[t]) ** 2 
                   for t in texp) 
    m.wssq = pyo.Objective(rule=wssq_rule)
   
    disc = pyo.TransformationFactory('dae.collocation')
    disc.apply_to(m, nfe=60, ncp=4)
    
    #disc = TransformationFactory('dae.finite_difference')
    #disc.apply_to(m, nfe=500, scheme = 'BACKWARD')
    
    return m 

In [24]:
# Estimate parameters
mest = hbv_model(data[0])
solver = pyo.SolverFactory('ipopt')
solver.solve(mest, tee = True)

Ipopt 3.14.5: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     8883
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2073

Total number of variables............................:     1449
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:     1443
Total number of inequ

 104  1.1224765e+04 2.72e-08 4.37e-03  -3.8 3.84e-03    -  1.00e+00 1.00e+00h  1
 105  1.1224765e+04 4.38e-11 8.69e-07  -3.8 2.93e-08    -  1.00e+00 1.00e+00h  1
 106  1.1224765e+04 8.61e-11 1.30e-05  -5.7 2.13e-04    -  1.00e+00 1.00e+00h  1
 107  1.1224765e+04 3.88e-11 2.53e-07  -8.6 2.64e-06    -  1.00e+00 1.00e+00h  1
 108  1.1224765e+04 3.10e-11 1.27e-07  -8.6 1.11e-11    -  1.00e+00 1.00e+00h  1
 109  1.1224765e+04 4.43e-11 6.38e-07  -8.6 2.15e-11    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 110  1.1224765e+04 3.03e-11 9.14e-07  -8.6 2.12e-11    -  1.00e+00 1.00e+00h  1
 111  1.1224765e+04 3.75e-11 2.40e-06  -8.6 1.76e-11    -  1.00e+00 1.00e+00h  1
 112  1.1224765e+04 5.22e-11 1.70e-06  -8.6 1.75e-11    -  1.00e+00 1.00e+00h  1
 113  1.1224765e+04 6.10e-11 2.43e-07  -8.6 1.61e-11    -  1.00e+00 1.00e+00H  1
 114  1.1224765e+04 4.63e-11 1.58e-07  -8.6 1.66e-11    -  1.00e+00 2.50e-01h  3
 115  1.1224765e+04 3.94e-11

{'Problem': [{'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 1443, 'Number of variables': 1449, 'Sense': 'unknown'}], 'Solver': [{'Status': 'ok', 'Message': 'Ipopt 3.14.5\\x3a Solved To Acceptable Level.', 'Termination condition': 'optimal', 'Id': 1, 'Error rc': 0, 'Time': 0.5557653903961182}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [25]:
# Parameter Estimates
parest = {'k1': mest.k1(), 'k2': mest.k2(), 'k3': mest.k3(), 'k4': mest.k4(), 'k5': mest.k5(), 'k6': mest.k6()}
parest

{'k1': 2.079141373344474,
 'k2': 0.036900154488831056,
 'k3': 331.42580439257085,
 'k4': 0.37864017390792787,
 'k5': 0.6298787589414175,
 'k6': 7.4335074895635704e-06}

In [27]:
lparest = {k: np.log10(v) for (k, v) in parest.items()}
lparest

{'k1': 0.31788402063303123,
 'k2': -1.4329718155887339,
 'k3': 2.5203863190169966,
 'k4': -0.4217733090702861,
 'k5': -0.20074303687964493,
 'k6': -5.128806216666511}

In [ ]:
fig, ax = plt.subplots()
ax.plot(list(mest.time), [mest.ca[t]() for t in mest.time])
ax.scatter(data[0]['ca_exp'].keys(), data[0]['ca_exp'].values())

In [ ]:
fig, ax = plt.subplots()
ax.plot(list(mest.time), [mest.cb[t]() for t in mest.time])
ax.scatter(data[0]['cb_exp'].keys(), data[0]['cb_exp'].values())

In [ ]:
fig, ax = plt.subplots()
ax.plot(list(mest.time), [mest.cc[t]() for t in mest.time])
ax.scatter(data[0]['cc_exp'].keys(), data[0]['cc_exp'].values())

In [39]:
solve_result, inv_red_hes = inv_reduced_hessian_barrier(mest, 
                    independent_variables= [mest.k1, mest.k2, mest.k3, mest.k4, mest.k5, mest.k6],
                    tee=True)

Ipopt 3.14.5: bound_relax_factor=0
honor_original_bounds=no


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.5, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:     8883
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     2073

Total number of variables............................:     1449
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints...

In [40]:
# Parameter Estimates
parest = {'k1': mest.k1(), 'k2': mest.k2(), 'k3': mest.k3(), 'k4': mest.k4(), 'k5': mest.k5(), 'k6': mest.k6()}
parest

{'k1': 2.079141373346096,
 'k2': 0.03690015448911893,
 'k3': 331.42580439379356,
 'k4': 0.37864017391097254,
 'k5': 0.6298787589437668,
 'k6': 7.433507489555274e-06}

In [41]:
inv_red_hes

array([[ 1.19917203e-04,  1.21540385e-05,  4.48027497e-02,
         1.39951869e-04,  7.73012419e-05, -3.90958376e-10],
       [ 1.21540385e-05,  2.34471089e-06,  8.06194046e-03,
         2.45279111e-05,  1.57227102e-05, -6.48949734e-11],
       [ 4.48027495e-02,  8.06194043e-03,  1.43255571e+02,
         8.54051567e-02,  2.77760320e-01, -2.70839724e-07],
       [ 1.39951869e-04,  2.45279111e-05,  8.54051570e-02,
         2.59590467e-04,  1.63924581e-04, -6.94962375e-10],
       [ 7.73012415e-05,  1.57227101e-05,  2.77760320e-01,
         1.63924581e-04,  5.43035858e-04, -5.06941812e-10],
       [-3.90958376e-10, -6.48949734e-11, -2.70839725e-07,
        -6.94962374e-10, -5.06941813e-10,  2.00018712e-15]])

In [42]:
n = len(data[0]['ca_exp']) + len(data[0]['cb_exp']) + len(data[0]['cc_exp'])
p = 6
s2 = mest.wssq() / (n - p)
print('n:', n, ' p:', p, ' s2:', s2)
# Covariance Matrix
cov = 2 * s2 * inv_red_hes
cov

n: 153  p: 6  s2: 76.3589441690889


array([[ 1.83135020e-02,  1.85613910e-03,  6.84218133e+00,
         2.13731538e-02,  1.18052824e-02, -5.97063376e-08],
       [ 1.85613910e-03,  3.58079296e-04,  1.23120252e+00,
         3.74585079e-03,  2.40113910e-03, -9.91062331e-09],
       [ 6.84218130e+00,  1.23120252e+00,  2.18776882e+04,
         1.30428952e+01,  4.24189695e+01, -4.13620708e-05],
       [ 2.13731538e-02,  3.74585079e-03,  1.30428952e+01,
         3.96441079e-02,  2.50342159e-02, -1.06133186e-07],
       [ 1.18052824e-02,  2.40113909e-03,  4.24189695e+01,
         2.50342158e-02,  8.29312895e-02, -7.74190831e-08],
       [-5.97063376e-08, -9.91062330e-09, -4.13620709e-05,
        -1.06133186e-07, -7.74190832e-08,  3.05464353e-13]])

In [43]:
parm_sd = np.sqrt(np.diag(cov))
conf_mult = np.sqrt(p * spstat.f.ppf(0.95, p, n - p))
print('parameter estimate')
print(parest)
print("conf multiplier:", conf_mult)
conf_int = conf_mult * parm_sd
print("confidence interval delta from nominal [k1, k2, k3, k4, k5, k6]")
print(conf_int)

parameter estimate
{'k1': 2.079141373346096, 'k2': 0.03690015448911893, 'k3': 331.42580439379356, 'k4': 0.37864017391097254, 'k5': 0.6298787589437668, 'k6': 7.433507489555274e-06}
conf multiplier: 3.600647920002456
confidence interval delta from nominal [k1, k2, k3, k4, k5, k6]
[4.87266279e-01 6.81350003e-02 5.32575730e+02 7.16918821e-01
 1.03690714e+00 1.99003596e-06]


In [44]:
print("confidence interval % delta from nominal [k1, k2, k3, k4, k5, k6]")
conf_int_pct = []
for (i, v) in enumerate(parest.items()):
    conf_int_pct.append(conf_int[i] / v[1] * 100.0)
print([np.round(x) for x in conf_int_pct])

confidence interval % delta from nominal [k1, k2, k3, k4, k5, k6]
[23.0, 185.0, 161.0, 189.0, 165.0, 27.0]


In [46]:
lcov_diag = [np.diag(cov)[i]/(v[1] * np.log(10))**2 for (i, v) in enumerate(parest.items())]
l_conf_int = [conf_mult * np.sqrt(x) for x in lcov_diag]
l_conf_int

[0.1017809846231017,
 0.801911403433836,
 0.697877768932466,
 0.8222949108273149,
 0.7149360796224724,
 0.11626565777971647]

In [47]:
lparest = {k: np.log10(v) for (k, v) in parest.items()}
lparest

{'k1': 0.31788402063337007,
 'k2': -1.4329718155853457,
 'k3': 2.520386319018599,
 'k4': -0.42177330906679394,
 'k5': -0.20074303687802514,
 'k6': -5.128806216666995}